In [1]:
from pathlib import Path
import pandas as pd
import os
import cdsapi
import torch
import xarray as xr
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from huggingface_hub import hf_hub_download

from aurora import Batch, Metadata, Aurora, rollout

In [7]:
DAY_START = pd.Timestamp("2025-10-13")
DAY_END = pd.Timestamp("2025-10-14")

DOWNLOAD_PATH = Path("../data/era5")
DOWNLOAD_PATH.mkdir(parents=True, exist_ok=True)

# Download all relevant ERA5 inputs for Aurora
Can download 4GB+ of data, depending on date range

In [8]:
c = cdsapi.Client(sleep_max=10)

def download_static():
    c.retrieve(
        "reanalysis-era5-single-levels",
        {
            "product_type": "reanalysis",
            "variable": [
                "geopotential",
                "land_sea_mask",
                "soil_type",
            ],
            "year": "2023", # doesn't matter, doesn't change
            "month": "01",
            "day": "01",
            "time": "00:00",
            "format": "netcdf",
        },
        str(DOWNLOAD_PATH / "static.nc"),
    )
    print("Static variables downloaded!")

def download_data(file_name: Path, data_source, vars, pressure_levels, time=["00:00", "06:00", "12:00", "18:00"]):
    params = {
            "product_type": "reanalysis",
            "variable": vars,
            "year": str(DAY_START.year),
            "month": str(DAY_START.month).zfill(2),
            "day": [str(date.day).zfill(2) for date in pd.date_range(DAY_START, DAY_END, freq="D")],
            "time": time,
            "format": "netcdf",
        }
    if pressure_levels:
        params["pressure_level"] = pressure_levels

    file_name.parent.mkdir(parents=True, exist_ok=True)
    c.retrieve(
        data_source,
        params,
        str(file_name),
    )

In [9]:
if not (DOWNLOAD_PATH / "static.nc").exists():
   download_static()

#Download the surface-level variables.
surface_path = DOWNLOAD_PATH / "surf_vars" / f"{DAY_START.strftime('%Y-%m-%d')}_{DAY_END.day}-surface-level.nc"
if not surface_path.exists():
    download_data(surface_path, "reanalysis-era5-single-levels", ["2m_temperature", "10m_u_component_of_wind", "10m_v_component_of_wind", "mean_sea_level_pressure"], None)
    print("Surface-level variables downloaded!")

# Download the atmospheric variables.
atmos_path = DOWNLOAD_PATH / "atmos_vars" / f"{DAY_START.strftime('%Y-%m-%d')}_{DAY_END.day}-atmospheric.nc"
if not atmos_path.exists():
    download_data(atmos_path, "reanalysis-era5-pressure-levels", 
                  ["temperature", "u_component_of_wind", "v_component_of_wind", "specific_humidity", "geopotential"], 
                  ["50", "100", "150", "200", "250", "300", "400", "500", "600", "700", "850", "925", "1000"])
    print("Atmospheric variables downloaded!")

2025-10-20 10:14:52,117 INFO Request ID is c9f81d91-ed7e-4794-b61f-1224a4ed0118
2025-10-20 10:14:52,352 INFO status has been updated to accepted
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attempt 1 of 500
Retrying in 10 seconds
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attempt 2 of 500
Retrying in 10 seconds
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attempt 3 of 500
Retrying in 10 seconds
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attempt 4 of 500
Retrying in 10 seconds
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attempt 5 of 500
Retrying in 10 seconds
2025-10-20

afad128a23c5c9d318d3f1ea512ee6fb.nc:   0%|          | 0.00/52.5M [00:00<?, ?B/s]

Surface-level variables downloaded!


2025-10-20 10:21:38,578 INFO Request ID is cd9b239f-9819-4a3e-88af-48537f7f1243
2025-10-20 10:21:38,756 INFO status has been updated to accepted
2025-10-20 10:21:44,089 INFO status has been updated to running
Recovering from connection error [('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))], attempt 1 of 500
Retrying in 10 seconds
2025-10-20 10:25:52,916 INFO status has been updated to successful


3e640c84dc6739939af7a7408c9a7a4f.nc:   0%|          | 0.00/826M [00:00<?, ?B/s]

Atmospheric variables downloaded!


# Convert data to batch for Aurora

In [10]:
static_vars_ds = xr.open_dataset(DOWNLOAD_PATH / "static.nc", engine="netcdf4")
surf_vars_ds = xr.open_dataset(surface_path, engine="netcdf4")
atmos_vars_ds = xr.open_dataset(atmos_path, engine="netcdf4")

In [ ]:
batch = Batch(
    surf_vars={
        # First select the first two time points: 00:00 and 06:00. Afterwards, `[None]`
        # inserts a batch dimension of size one.
        "2t": torch.from_numpy(surf_vars_ds["t2m"].values[:2][None]),
        "10u": torch.from_numpy(surf_vars_ds["u10"].values[:2][None]),
        "10v": torch.from_numpy(surf_vars_ds["v10"].values[:2][None]),
        "msl": torch.from_numpy(surf_vars_ds["msl"].values[:2][None]),
    },
    static_vars={
        # The static variables are constant, so we just get them for the first time.
        "z": torch.from_numpy(static_vars_ds["z"].values[0]),
        "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
        "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
    },
    atmos_vars={
        "t": torch.from_numpy(atmos_vars_ds["t"].values[:2][None]),
        "u": torch.from_numpy(atmos_vars_ds["u"].values[:2][None]),
        "v": torch.from_numpy(atmos_vars_ds["v"].values[:2][None]),
        "q": torch.from_numpy(atmos_vars_ds["q"].values[:2][None]),
        "z": torch.from_numpy(atmos_vars_ds["z"].values[:2][None]),
    },
    metadata=Metadata(
        lat=torch.from_numpy(surf_vars_ds.latitude.values),
        lon=torch.from_numpy(surf_vars_ds.longitude.values),
        # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
        # `datetime.datetime`s. Note that this needs to be a tuple of length one:
        # one value for every batch element. Select element 1, corresponding to time
        # 06:00.
        time=(surf_vars_ds.valid_time.values.astype("datetime64[s]").tolist()[1],),
        atmos_levels=tuple(int(level) for level in atmos_vars_ds.pressure_level.values),
    ),
)

# Setup Aurora Model & Inference

In [12]:
model = Aurora(use_lora=False)  # The pretrained version does not use LoRA.
#model_path = hf_hub_download(repo_id=model.default_checkpoint_repo, filename="aurora-0.25-pretrained.ckpt", cache_dir="/workspace/aurora/model/")
#model.load_checkpoint_local(model_path)

model.eval()
model = model.to("cuda")
print("Num parameters: ", sum([p.numel() for p in model.parameters()]))

Num parameters:  1256300176


In [13]:
with torch.inference_mode():
        steps = surf_vars_ds["t2m"].shape[0] - 2
        preds = [pred.to("cpu") for pred in rollout(model, batch, steps=steps)]

In [22]:
len(preds)

6

In [ ]:
import sys
sys.path.append("../")
from backend.prediction import Prediction
from scipy.interpolate import PchipInterpolator

def hermit_interp(data: np.ndarray) -> np.ndarray:
        """
        Interpolates a tensor with 6-hourly timestamps to hourly resolution
        using Piecewise Cubic Hermite Interpolating Polynomial (PCHIP).

        Args:
            data_6hr (np.ndarray): The input data tensor with shape (x, 721, 1440).

        Returns:
            np.ndarray: The interpolated data tensor with shape (6(x-1), 721, 1440).
        """
        # Original times: 0, 6, 12, ..., 246 hours (41 * 6)
        t_original = np.arange(data.shape[0]) * 6

        # New times: 0, 1, 2, ..., 246 hours.
        t_new = np.arange(t_original[-1] + 1)

        interpolator = PchipInterpolator(t_original, data, axis=0)

        data_hourly = interpolator(t_new)
        # remove first datapoint (only there for interpolation)
        return data_hourly[1:]

# NOTE: Works
def _post_process_generic(data, add_first_row=True):
        # aurora post-process
        # Make latitude go from -87.5 to 90.0 instead of decreasing order
        # duplicate the first row for a -90.0 prediction
        data = data.squeeze().flip(dims=(-2,))
        if add_first_row:
            data = torch.cat((data[-2].unsqueeze(0), data))
        return data

def compute_surface_pressure(batch: 'Batch'):
    # --- Physical Constants ---
    G0 = 9.80665    # Standard gravity (m/s^2)
    R_AIR = 287.05  # Gas constant for dry air (J/(kg*K))

    # P_msl is typically in Pascals (Pa)
    P_msl = batch.surf_vars["msl"][0, -1]  
    T_2m = batch.surf_vars["2t"][0, -1]    # 2m Temperature (K)
    z_sfc = batch.static_vars["z"]         # Surface Geopotential (m^2/s^2)
    
    h_sfc = z_sfc / G0  # h_sfc in meters (m)

    exponent = (G0 * h_sfc) / (R_AIR * T_2m)
    
    # Calculate P_sfc
    P_sfc = P_msl * torch.exp(-exponent) # P_sfc in Pascals (Pa)
    
    return P_sfc.unsqueeze(0)


def predict(batch: Batch, preds) -> Prediction:
        # also include last of batch, so that interpolation for first 6 hours is possible
        temp_preds = [_post_process_generic(batch.surf_vars["2t"][:, -1], add_first_row=False)]
        east_wind_preds = [_post_process_generic(get_100m_wind(batch, "u"), add_first_row=False)]
        north_wind_preds = [_post_process_generic(get_100m_wind(batch, "v"), add_first_row=False)]


        for pred in preds:
            temp_preds.append(_post_process_generic(pred.surf_vars["2t"]))
            east_wind_preds.append(_post_process_generic(get_100m_wind(pred, "u")))
            north_wind_preds.append(_post_process_generic(get_100m_wind(pred, "v")))     

        return Prediction(
            temperature = hermit_interp(torch.stack(temp_preds, dim=0).numpy()),
            u_wind_100m = hermit_interp(torch.stack(east_wind_preds, dim=0).numpy()),
            v_wind_100m = hermit_interp(torch.stack(north_wind_preds, dim=0).numpy())
        )   

In [207]:
gt = xr.open_dataset("../data/era5/era5_t2m_2025-10-13.nc", engine="netcdf4")

In [209]:
gt_data = _post_process_generic(torch.Tensor(gt["t2m"][6].to_numpy()), add_first_row=False)

In [203]:
conv_data = _post_process_generic(compute_surface_pressure(batch), add_first_row=False)
#_post_process_generic(batch.surf_vars["10u"][:, -1], add_first_row=False)compute_surface_pressure(batch)

In [204]:
((gt_data - conv_data) ** 2).mean().sqrt()

tensor(641.2620)

In [131]:
batch.metadata.time

(datetime.datetime(2025, 10, 13, 6, 0),)

In [52]:
atmos_vars_ds.pressure_level.values

array([1000.,  925.,  850.,  700.,  600.,  500.,  400.,  300.,  250.,
        200.,  150.,  100.,   50.])

In [119]:
4.92 * np.sin(np.deg2rad(114))

np.float64(4.494643651601597)

In [39]:
batch.metadata.time

(datetime.datetime(2025, 10, 13, 6, 0),)